<a href="https://colab.research.google.com/github/mateopolancecofficial/NLP/blob/main/WineReviews/SequenceModelText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pip install -q -U tensorflow-text

In [3]:
pip install -q -U tf-models-official

In [4]:
#pip install -U tfds-nightly

In [7]:
!git clone -l -s https://github.com/mateopolancecofficial/NLP.git

Cloning into 'NLP'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 33 (delta 8), reused 21 (delta 5), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [4]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text  # A dependency of the preprocessing model
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np

tf.get_logger().setLevel('ERROR')

In [5]:
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [6]:
import os

if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.test.is_gpu_available():
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')

Using TPU


In [7]:
df = pd.read_csv("/content/NLP/WineReviews/data/winemag-data-130k-v2.csv", index_col=0)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [8]:
df = df[['description', 'points']]
df.head()

,description,points
0,"Aromas include tropical fruit, broom, brimston...",87
1,"This is ripe and fruity, a wine that is smooth...",87
2,"Tart and snappy, the flavors of lime flesh and...",87
3,"Pineapple rind, lemon pith and orange blossom ...",87
4,"Much like the regular bottling from 2012, this...",87


In [9]:
df = df.sample(frac = 1)
df.head()

,description,points
56537,Here's a pleasantly fragrant blend of Negroama...,87
492,"Tart and lively, this medium-bodied Sangiovese...",87
112903,The latest release from Vallado is a value win...,87
127846,"The wine has good acidity, crisp green and whi...",87
84888,"The wine is dry, although its soft and creamy ...",88


In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt



In [11]:
df.description = df.description.str.replace('\d+', '')

In [12]:
from sklearn.model_selection import train_test_split

text_train, text_val, y_train, y_val = train_test_split(df.description, df.points,
                                                  test_size=0.2,train_size=0.8)

In [13]:
tf.config.run_functions_eagerly(False)

def create_model():
  hub_model = "https://tfhub.dev/google/nnlm-en-dim50/2"
  hub_layer = hub.KerasLayer(hub_model, input_shape=[], dtype=tf.string, trainable=False)
  model = tf.keras.Sequential()
  model.add(hub_layer)
  #model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(8, activation='relu'))
  #model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(1))
  return model


In [17]:

#with strategy.scope():
loss = 'mean_squared_error'
metric = 'mean_squared_error'
model = create_model()
model.compile(optimizer='adam',
              loss=loss,
              metrics=[metric])
  

In [18]:
checkpoint_path = "/content/NLP/WineReviews/checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 64

AUTOTUNE = tf.data.AUTOTUNE
seed = 42

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

train_dataset = tf.data.Dataset.from_tensor_slices((text_train, y_train))
train_dataset = train_dataset.batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((text_val, y_val))
val_dataset = val_dataset.batch(batch_size)

train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [23]:
# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=1,
                    )

1625/1625 [==============================] - 20s 13ms/step - loss: 58.5678 - mean_squared_error: 58.5678 - val_loss: 42.2012 - val_mean_squared_error: 42.2012


In [24]:
model.save_weights(checkpoint_path.format(epoch=0))

In [26]:


model.save_weights('./saved_weights.h5', overwrite=True)

# Load the previously saved weights
#model.load_weights(latest)

In [29]:
model.load_weights("./saved_weights.h5")